# Machine Intelligence with Deep Learning
## Importance batching for improved training of neural networks
---

In [1]:
import timeit

In [2]:
from models.resnet import ResNet18
from utils.data_utils import DataLoader
from utils.logging_utils import *

import numpy as np
import torch
import torch.nn as nn
import torch.optim as optim
import torch.nn.functional as F
import torch.backends.cudnn as cudnn

import os
import pandas as pd

from datetime import datetime
today = datetime.today().strftime('%Y%m%d')

In [3]:
SEEDS = [10, 42, 4] # don't change!
#STRATEGIES = ['freeze', 'shuffle', 'homogeneous', 'heterogeneous', 'max_k_loss', 'min_k_loss'] # can be changed
STRATEGIES = ['max_k_loss']
#['weighted_random_sampling']#, 'max_k_loss']#, 'heterogeneous_max_k_loss', 'heterogeneous_min_k_loss']
UPDATE_STRATEGIES = ['on_test_acc_reduction', 'every_epoch', 'every_iteration']
update_strategy = UPDATE_STRATEGIES[0] # current implementation: choose one during testing

In [4]:
### Training
def train(epoch, optimizer, criterion_fn, seed, dataloader, strategy, device, update_strategy):
    criterion = criterion_fn()
    net.train()
    train_loss = 0
    correct = 0
    total = 0
    update_every_iteration = True if update_strategy == 'every_iteration' else False
    for batch_idx, (inputs, targets) in enumerate(dataloader.yield_batches(strategy, \
                                                  random_state=seed, use_train=True, \
                                                  criterion=criterion_fn(reduction='none'),\
                                                  device=device, num_iterations=500,
                                                  update_every_iteration=update_every_iteration)): # 500: 50,000 images / 100 batch size
        inputs, targets = inputs.to(device), targets.to(device)
        optimizer.zero_grad()
        outputs = net(inputs)
        loss = criterion(outputs, targets)
        loss.backward()
        optimizer.step()
        
    with torch.no_grad():
        for batch_idx, (inputs, targets) in enumerate(dataloader.yield_batches('freeze', \
                                                                               random_state=seed, use_train=True)):
            inputs, targets = inputs.to(device), targets.to(device)
            outputs = net(inputs)
            loss = criterion(outputs, targets)
            train_loss += loss.item()
            _, predicted = outputs.max(1)
            total += targets.size(0)
            correct += predicted.eq(targets).sum().item()
    train_acc = 100.*correct/total
    train_loss /= total
    return train_acc, train_loss

### Testing
def test(epoch, best_acc, criterion_fn, seed, dataloader, strategy, device, last_acc, update_strategy):
    criterion = criterion_fn()
    net.eval()
    test_loss = 0
    correct = 0
    total = 0
    with torch.no_grad():
        for batch_idx, (inputs, targets) in enumerate(dataloader.yield_batches('freeze', \
                                                                               random_state=seed, use_train=False)):
            inputs, targets = inputs.to(device), targets.to(device)
            outputs = net(inputs)
            loss = criterion(outputs, targets)

            test_loss += loss.item()
            _, predicted = outputs.max(1)
            total += targets.size(0)
            correct += predicted.eq(targets).sum().item()

    # Save checkpoint.
    test_acc = 100.*correct/total
    test_loss /= total
    if update_strategy == 'on_test_acc_reduction':
        print(test_acc, last_acc)
        if test_acc < last_acc:
            print("UPDATING DATALOADER")
            if strategy in ['max_k_loss', 'min_k_loss', 'weighted_random_sampling']:
                dataloader.initialize_weights(criterion_fn(reduction='none'), device, seed=seed, 
                                              dump='./dump_logs/{}_{}_{}.txt'.format(strategy, seed, epoch))
            if strategy in ['heterogeneous_max_k_loss', 'heterogeneous_min_k_loss']:
                dataloader.initialize_weights_per_class(criterion_fn(reduction='none'), device, seed=seed)
    if test_acc > best_acc:
        best_acc = test_acc
        net.save(best_acc, epoch, seed, strategy)
    return test_acc, test_loss, dataloader

In [5]:
resume = False
given_date = '20191113' #only needed if resumed from checkpoint
device = 'cuda' if torch.cuda.is_available() else 'cpu'
if device == 'cuda':
    torch.backends.cudnn.deterministic = True
    torch.backends.cudnn.benchmark = False
    
### task: classification of the following classes
classes = ['plane', 'car', 'bird', 'cat', 'deer', 'dog', 'frog', 'horse', 'ship', 'truck']

    
### hyperparameters
test_acc = 0  # best test accuracy
start_epoch = 0  # start from epoch 0 or last checkpoint epoch
num_epochs = 100 # number of epochs the model gets trained
learning_rate = 0.01
momentum = 0.9
batch_size = 100
weight_decay = 5e-4

print("Begin training.")
start = timeit.default_timer()
#Logging header
length_table = 90
log_separating_line(length_table)
log_header_line("Seeds: {}".format(SEEDS), length_table)
log_header_line("Strategies: {}".format(STRATEGIES), length_table)
log_header_line("-> Resulting number of iterations: {}".format(len(SEEDS) * len(STRATEGIES)), length_table)
log_header_line("Number of iterations: {}".format(num_epochs), length_table)
log_header_line("Learning rate: {}".format(learning_rate), length_table)
log_header_line("Resuming from checkpoint: {}".format(True if resume else False), length_table)
log_separating_line(length_table)

rows = []
for seed in SEEDS:
    for strategy in STRATEGIES:
        np.random.seed(seed)
        torch.manual_seed(seed)
        if device == 'cuda':
            torch.cuda.manual_seed_all(seed)
             
        ### Model
        net = ResNet18()
        net = net.to(device)
        criterion = nn.CrossEntropyLoss # no function!
        optimizer = optim.SGD(net.parameters(), lr=learning_rate, momentum=momentum, weight_decay=weight_decay)
        
        ### load the data
        # if needed, specify batch sizes and shuffle settings
        dataloader = DataLoader(batch_size=batch_size)
        dataloader.download_cifar()
        dataloader.set_model(net)
        
        if strategy in ['max_k_loss', 'min_k_loss', 'weighted_random_sampling']:
            dataloader.initialize_weights(criterion(reduction='none'), device, seed=seed, 
                                          dump='./dump_logs/{}_{}_{}.txt'.format(strategy, seed, 0))
        if strategy in ['heterogeneous_max_k_loss', 'heterogeneous_min_k_loss']:
            dataloader.initialize_weights_per_class(criterion(reduction='none'), device, seed=seed)
        last_acc = 0
            
        print()
        log_separating_line(length_table)
        log_position_header(seed, strategy, length_table)
        log_separating_line(length_table)

        if resume:
            assert os.path.isdir('serialized'), 'Error: no serialized directory found!'
            ckpt = torch.load('./serialized/{}/{}_ckpt_{}.pth'.format(given_date, strategy, seed))
            test_acc, start_epoch, net = net.load(ckpt)

        for epoch in range(start_epoch, start_epoch+num_epochs):
            if update_strategy == 'every_epoch':
                if strategy in ['max_k_loss', 'min_k_loss', 'weighted_random_sampling']:
                    dataloader.initialize_weights(criterion(reduction='none'), device, seed=seed, 
                                                  dump='./dump_logs/{}_{}_{}.txt'.format(strategy, seed, epoch))
                if strategy in ['heterogeneous_max_k_loss', 'heterogeneous_min_k_loss']:
                    dataloader.initialize_weights_per_class(criterion(reduction='none'), device, seed=seed)

            train_acc, train_loss = train(epoch, optimizer, criterion, seed, dataloader, \
                                          strategy, device, update_strategy)
            test_acc, test_loss, dataloader = test(epoch, test_acc, criterion, seed, dataloader, \
                                                   strategy, device, last_acc, update_strategy)
            last_acc = test_acc
            log_position_line(epoch + 1, num_epochs, train_acc, test_acc, train_loss, test_loss, length_table)
            row = {
                'epoch': epoch + 1,
                'seed': seed,
                'train': True,
                'strategy': strategy,
                'accuracy': train_acc,
                'loss': train_loss
            }
            rows.append(row)
            row = {
                'epoch': epoch + 1,
                'seed': seed,
                'train': False,
                'strategy': strategy,
                'accuracy': test_acc,
                'loss': test_loss
            }
            rows.append(row)
        log_separating_line(length_table)
            
stop = timeit.default_timer()
time_needed = stop - start
hrs = int(time_needed / 3600)
mins = int((time_needed / 60) % 60)
secs = int(time_needed % 60)
print()
print("Finished training. Time needed: {} hrs {} mins {} secs".format(hrs, mins, secs))

logging_df = pd.DataFrame(rows, columns=['epoch', 'seed', 'train', 'strategy', 'accuracy', 'loss'])   
training_logs_dir = 'evaluation_logs'
logging_df.to_csv('{}.txt'.format(os.path.join(training_logs_dir, today)), sep='\t', index=False)

Begin training.
+----------------------------------------------------------------------------------------+
| Seeds: [10, 42, 4]                                                                     |
| Strategies: ['max_k_loss']                                                             |
| -> Resulting number of iterations: 3                                                   |
| Number of iterations: 100                                                              |
| Learning rate: 0.01                                                                    |
| Resuming from checkpoint: False                                                        |
+----------------------------------------------------------------------------------------+
Files already downloaded and verified
Files already downloaded and verified

+----------------------------------------------------------------------------------------+
| Seed: 10      Strategy: max_k_loss                                                    

77.37 75.91
| [064/100]:    97.87           77.37           0.00065      0.00999                     |
76.77 77.37
UPDATING DATALOADER
| [065/100]:    96.02           76.77           0.00115      0.01056                     |
77.63 76.77
| [066/100]:    098.4           77.63           0.00049      0.01006                     |
78.03 77.63
| [067/100]:    97.17           78.03           00.0008      00.0097                     |
60.72 78.03
UPDATING DATALOADER
| [068/100]:    76.91           60.72           0.00767      0.02023                     |
80.68 60.72
| [069/100]:    99.56           80.68           0.00018      0.00769                     |
74.69 80.68
UPDATING DATALOADER
| [070/100]:    95.74           74.69           0.00128      0.01185                     |
79.82 74.69
| [071/100]:    99.05           79.82           00.0003      0.00855                     |
77.35 79.82
UPDATING DATALOADER
| [072/100]:    98.06           77.35           0.00058      0.01008                

78.97 79.15
UPDATING DATALOADER
| [034/100]:    99.16           78.97           0.00026      0.01076                     |
79.83 78.97
| [035/100]:    99.42           79.83           0.00019      0000.01                     |
77.11 79.83
UPDATING DATALOADER
| [036/100]:    98.65           77.11           00.0004      0.01159                     |
78.59 77.11
| [037/100]:    98.63           78.59           0.00041      0.01038                     |
76.41 78.59
UPDATING DATALOADER
| [038/100]:    97.69           76.41           0.00067      00.0114                     |
78.78 76.41
| [039/100]:    99.06           78.78           0.00028      0.00992                     |
77.3 78.78
UPDATING DATALOADER
| [040/100]:    98.21           077.3           0.00052      0.01129                     |
78.56 77.3
| [041/100]:    98.96           78.56           0.00031      00.0101                     |
78.27 78.56
UPDATING DATALOADER
| [042/100]:    98.87           78.27           0.00034      0.010

38.63 35.63
| [002/100]:    40.32           38.63           0.01718      00.0171                     |
37.41 38.63
UPDATING DATALOADER
| [003/100]:    38.74           37.41           0.01712      0.01706                     |
43.36 37.41
| [004/100]:    48.16           43.36           0.01419      00.0151                     |
46.78 43.36
| [005/100]:    50.18           46.78           0.01332      00.0139                     |
53.27 46.78
| [006/100]:    58.39           53.27           0.01095      0.01227                     |
61.9 53.27
| [007/100]:    65.14           061.9           0.00939      0.01034                     |
57.79 61.9
UPDATING DATALOADER
| [008/100]:    63.75           57.79           00.0095      0.01179                     |
66.77 57.79
| [009/100]:    074.5           66.77           0.00697      0.00954                     |
68.82 66.77
| [010/100]:    77.38           68.82           0.00621      0.00937                     |
68.11 68.82
UPDATING DATALOADER
| [

78.08 79.12
UPDATING DATALOADER
| [076/100]:    98.37           78.08           00.0005      0.00958                     |
79.29 78.08
| [077/100]:    98.85           79.29           0.00037      0.00883                     |
76.54 79.29
UPDATING DATALOADER
| [078/100]:    96.97           76.54           0.00092      0.01041                     |
79.07 76.54
| [079/100]:    98.64           79.07           0.00042      0.00915                     |
77.41 79.07
UPDATING DATALOADER
| [080/100]:    97.33           77.41           0.00077      0.01008                     |
77.65 77.41
| [081/100]:    97.84           77.65           0.00068      0.00952                     |
75.55 77.65
UPDATING DATALOADER
| [082/100]:    96.33           75.55           0.00105      0.01131                     |
78.84 75.55
| [083/100]:    99.33           78.84           0.00023      0.00877                     |
76.49 78.84
UPDATING DATALOADER
| [084/100]:    97.59           76.49           0.00072      0.0